In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=106421/Daniil-Medvedev&table=mcp-serve'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='mcp-serve')
print(table.prettify())

driver.quit()


<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="mcp-serve">
 <thead>
  <tr>
   <th align="left">
    <span title="">
     Match
    </span>
   </th>
   <th align="left">
    <span title="">
     Result
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of serves that
were unreturned">
     Unret%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of service points
won on either the serve or second shot">
     &lt;=3 W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of points won when
return was put in play">
     RiP W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Serve Impact: Advanced stat
estimating how many service points
were won due to the serve">
     SvImpact
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of first serves
that were unreturned">
     1st: Unret%
    </span>
   </th>
   <th a

In [2]:
data = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            data.append((cell.get_text(strip=True)))


2025 Madrid QF
L vsRuud
15.4%
33.3%
57.4%
26.3%
17.4%
41.3%
60.5%
35.9%
54.2%
77.3%
100.0%
14.8%
25.9%
52.2%
19
2025 Monte Carlo R16
L vsDe Minaur
11.5%
13.5%
30.2%
13.7%
20.7%
24.1%
30.4%
24.5%
40.0%
21.4%
33.3%
0.0%
0.0%
30.0%
-118
2025 Miami R64
L vsMunar
18.9%
34.0%
48.7%
28.1%
22.6%
38.7%
50.0%
38.4%
75.0%
60.0%
100.0%
16.7%
33.3%
46.7%
32
2025 Indian Wells SF
L vsRune
20.0%
26.7%
50.0%
24.5%
23.3%
33.3%
56.5%
32.3%
46.7%
20.0%
0.0%
16.7%
20.0%
44.0%
-79
2025 Indian Wells QF
W vsFils
27.3%
34.1%
57.4%
30.1%
32.7%
41.8%
45.9%
37.3%
77.3%
54.5%
33.3%
20.0%
23.3%
75.0%
59
2025 Dubai QF
L vsGriekspoor
33.9%
42.2%
54.4%
39.3%
35.9%
46.2%
54.0%
43.3%
65.7%
37.2%
50.0%
33.3%
37.0%
55.6%
116
2025 Dubai R16
W vsMpetshi Perricard
39.6%
50.0%
75.0%
41.9%
54.2%
58.3%
90.9%
58.8%
72.7%
69.2%
-
26.1%
43.5%
64.7%
-16
2025 Doha R16
W vsBergs
33.3%
36.1%
72.7%
38.1%
40.7%
44.4%
75.0%
47.0%
58.3%
53.3%
-
14.3%
14.3%
66.7%
4
2025 Marseille R16
W vsHerbert
44.9%
55.1%
70.8%
49.2%
51.7%
62.1%
78.6%
59

In [3]:
data = [d.replace('\xa0',' ') for d in data]

In [4]:
year_tourney_round = data[::17] # year tournament round in a single string

In [5]:
df_rows = []
for i in range(0, len(data)):
    if i % 17 == 0:
        row = data[i:i+17]
        df_rows.append(row)

In [6]:
cols = ['year tourney round', 'result opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%',
        '1st: <=3 W%', '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%', '2nd: Unret%', '2nd: <=3 W%',
        '2nd: RiP W%', '2ndAgg']
df = pd.DataFrame(df_rows, columns=cols)
df['year'] = df['year tourney round'].apply(lambda x: x.split(' ')[0])
df['tourney'] = df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
df['round'] = df['year tourney round'].apply(lambda x: x.split(' ')[-1]) 
df['result'] = df['result opponent'].apply(lambda x: x.split(' ')[0])
df['opponent'] = df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
df.head(10)

,overall: Unret%,overall: <=3 W%,overall: RiP W%,overall: SvImpact,1st: Unret%,1st: <=3 W%,1st: RiP W%,1st: SvImpact,1st: D Wide%,1st: A Wide%,1st: A BP Wide%,2nd: Unret%,2nd: <=3 W%,2nd: RiP W%,2ndAgg,year,tourney,round,result,opponent
0,15.4%,33.3%,57.4%,26.3%,17.4%,41.3%,60.5%,35.9%,54.2%,77.3%,100.0%,14.8%,25.9%,52.2%,19,2025,Madrid,QF,L,Ruud
1,11.5%,13.5%,30.2%,13.7%,20.7%,24.1%,30.4%,24.5%,40.0%,21.4%,33.3%,0.0%,0.0%,30.0%,-118,2025,Monte Carlo,R16,L,De
2,18.9%,34.0%,48.7%,28.1%,22.6%,38.7%,50.0%,38.4%,75.0%,60.0%,100.0%,16.7%,33.3%,46.7%,32,2025,Miami,R64,L,Munar
3,20.0%,26.7%,50.0%,24.5%,23.3%,33.3%,56.5%,32.3%,46.7%,20.0%,0.0%,16.7%,20.0%,44.0%,-79,2025,Indian Wells,SF,L,Rune
4,27.3%,34.1%,57.4%,30.1%,32.7%,41.8%,45.9%,37.3%,77.3%,54.5%,33.3%,20.0%,23.3%,75.0%,59,2025,Indian Wells,QF,W,Fils
5,33.9%,42.2%,54.4%,39.3%,35.9%,46.2%,54.0%,43.3%,65.7%,37.2%,50.0%,33.3%,37.0%,55.6%,116,2025,Dubai,QF,L,Griekspoor
6,39.6%,50.0%,75.0%,41.9%,54.2%,58.3%,90.9%,58.8%,72.7%,69.2%,-,26.1%,43.5%,64.7%,-16,2025,Dubai,R16,W,Mpetshi
7,33.3%,36.1%,72.7%,38.1%,40.7%,44.4%,75.0%,47.0%,58.3%,53.3%,-,14.3%,14.3%,66.7%,4,2025,Doha,R16,W,Bergs
8,44.9%,55.1%,70.8%,49.2%,51.7%,62.1%,78.6%,59.0%,28.6%,46.7%,100.0%,41.2%,52.9%,60.0%,148,2025,Marseille,R16,W,Herbert
9,28.0%,41.9%,59.3%,35.9%,40.4%,59.6%,74.2%,54.6%,68.2%,40.0%,33.3%,15.2%,24.2%,42.9%,16,2025,Rotterdam,R16,L,Bellucci


In [7]:
neworder = ['year','tourney', 'round', 'result', 'opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%', '1st: <=3 W%',
       '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%',
       '2nd: Unret%', '2nd: <=3 W%', '2nd: RiP W%', '2ndAgg']

df = df[neworder]

In [8]:

df.to_csv('../data/medvedev_serve_chart.csv', index=False) #save to csv in data folder

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=106421/Daniil-Medvedev&table=serve-speed'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='serve-speed')
print(table.prettify())

driver.quit()

<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="serve-speed">
 <thead>
  <tr>
   <th align="left">
    Match
   </th>
   <th align="left">
    Result
   </th>
   <th align="right" class="header">
    <span title="Average serve speed">
     Avg Speed
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average 1st serve speed">
     1st Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Standard deviation of
1st serve speed">
     1st StDev
    </span>
   </th>
   <th align="right" class="header">
    <span t""="" title="Average speed of 1st serves
down the ">
     1st T Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average speed of
1st serves hit wide">
     1st Wide Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Fastest second serve">
     Max 1st
    </span>
   </th>
   <th align="right" class="header">
    <span title="Slowest second serve">
  

In [10]:
speed = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            speed.append((cell.get_text(strip=True)))


2024 US Open QF
L vsSinner
105.4
114.5
9.7
119.3
109.8
126
92
92.2
6.3
91.0
89.2
107
79
2024 US Open R16
W vsBorges
111.2
118.8
7.5
121.6
113.0
129
102
95.2
4.9
103.0
95.8
107
84
2024 US Open R32
W vsCobolli
107.9
119.1
7.1
122.2
115.3
127
96
90.7
6.9
103.0
92.8
111
80
2024 US Open R64
W vsMarozsan
105.9
117.5
6.2
120.5
113.2
126
102
93.0
5.0
94.3
92.3
103
81
2024 US Open R128
W vsLajovic
108.1
117.5
7.1
120.3
112.9
127
86
93.1
4.1
98.0
91.1
104
85
2024 Wimbledon SF
L vsAlcaraz
112.8
117.7
4.2
120.0
115.7
124
107
100.2
11.5
114.1
97.3
124
82
2024 Wimbledon QF
W vsSinner
112.8
119.7
5.1
122.5
116.6
128
95
94.2
10.8
88.0
95.4
123
75
2024 Wimbledon R16
W vsDimitrov
107.2
116.0
5.3
117.4
112.8
122
103
91.9
7.8
108.0
88.0
108
84
2024 Wimbledon R32
W vsStruff
111.2
115.7
5.8
119.4
110.4
125
96
97.3
9.1
108.0
99.3
122
85
2024 Wimbledon R64
W vsMuller
108.8
116.5
7.0
119.9
111.8
127
76
93.7
6.3
95.8
95.6
105
81
2024 Wimbledon R128
W vsKovacevic
110.4
116.5
5.9
118.8
113.8
125
105
96.9
9.0

95.

In [11]:
speed = [s.replace('\xa0',' ') for s in speed]

In [12]:
year_tourney_round = speed[::15]

In [13]:
speed_df_rows = []
for i in range(0, len(speed)):
    if i % 15 == 0:
        row = speed[i:i+15]
        speed_df_rows.append(row)

In [14]:
cols = ['year tourney round', 'result opponent', 'avg speed', '1st: avg', '1st: stdev', '1st: t avg', '1st: wide avg', '1st: max', 
     '1st: min', '2nd: avg', '2nd: stdev', '2nd: t avg', '2nd: wide avg', '2nd: max', '2nd: min']

speed_df = pd.DataFrame(speed_df_rows, columns=cols)
speed_df['year'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[0])
speed_df['tourney'] = speed_df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
speed_df['round'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[-1])
speed_df['result'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[0])
speed_df['opponent'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
speed_df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
speed_df.head(10)

,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min,year,tourney,round,result,opponent
0,105.4,114.5,9.7,119.3,109.8,126,92,92.2,6.3,91.0,89.2,107,79,2024,US Open,QF,L,Sinner
1,111.2,118.8,7.5,121.6,113.0,129,102,95.2,4.9,103.0,95.8,107,84,2024,US Open,R16,W,Borges
2,107.9,119.1,7.1,122.2,115.3,127,96,90.7,6.9,103.0,92.8,111,80,2024,US Open,R32,W,Cobolli
3,105.9,117.5,6.2,120.5,113.2,126,102,93.0,5.0,94.3,92.3,103,81,2024,US Open,R64,W,Marozsan
4,108.1,117.5,7.1,120.3,112.9,127,86,93.1,4.1,98.0,91.1,104,85,2024,US Open,R128,W,Lajovic
5,112.8,117.7,4.2,120.0,115.7,124,107,100.2,11.5,114.1,97.3,124,82,2024,Wimbledon,SF,L,Alcaraz
6,112.8,119.7,5.1,122.5,116.6,128,95,94.2,10.8,88.0,95.4,123,75,2024,Wimbledon,QF,W,Sinner
7,107.2,116.0,5.3,117.4,112.8,122,103,91.9,7.8,108.0,88.0,108,84,2024,Wimbledon,R16,W,Dimitrov
8,111.2,115.7,5.8,119.4,110.4,125,96,97.3,9.1,108.0,99.3,122,85,2024,Wimbledon,R32,W,Struff
9,108.8,116.5,7.0,119.9,111.8,127,76,93.7,6.3,95.8,95.6,105,81,2024,Wimbledon,R64,W,Muller


In [15]:
neworder = ['year','tourney', 'round', 'result', 'opponent'] + cols[2:]
speed_df = speed_df[neworder]


In [16]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].replace('', np.nan)
speed_df

,year,tourney,round,result,opponent,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min
0,2024,US Open,QF,L,Sinner,105.4,114.5,9.7,119.3,109.8,126,92,92.2,6.3,91.0,89.2,107,79
1,2024,US Open,R16,W,Borges,111.2,118.8,7.5,121.6,113.0,129,102,95.2,4.9,103.0,95.8,107,84
2,2024,US Open,R32,W,Cobolli,107.9,119.1,7.1,122.2,115.3,127,96,90.7,6.9,103.0,92.8,111,80
3,2024,US Open,R64,W,Marozsan,105.9,117.5,6.2,120.5,113.2,126,102,93.0,5.0,94.3,92.3,103,81
4,2024,US Open,R128,W,Lajovic,108.1,117.5,7.1,120.3,112.9,127,86,93.1,4.1,98.0,91.1,104,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2018,US Open,R64,W,Tsitsipas,104.4,116.6,7.7,122.8,110.8,127,89,87.7,6.3,NaN,84.3,106,78
58,2018,Wimbledon,R32,L,Mannarino,101.6,112.9,12.4,118.6,107.3,129,86,88.3,6.4,89.0,86.7,103,77
59,2018,Wimbledon,R128,W,Coric,102.0,115.1,8.1,120.0,114.4,127,94,89.3,8.9,91.4,92.5,107,77
60,2017,Wimbledon,R128,W,Wawrinka,112.3,119.5,6.0,122.0,116.0,129,102,97.3,11.6,108.0,95.3,125,83


In [17]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].astype(float)

In [18]:
speed_df.to_csv('../data/medvedev_serve_speed.csv', index=False) #change the name of the file to save it as you want